## Band Math
Band math deals with performing mathematical operations on bands of a raster. 
These operations could involve single band or multiple bands of the same raster. 
The results of operations are analyzed to draw conclusions. 

Let's look at some mathematical operations involving single band and multiple bands. 

### Single Band
This section will demonstrate the usefulness of band math involving a single band. 
Before applying any algorithm on raster images, 
it is always good to know the statistics of each band, 
such as minimum, maximum and median values of the band. 
Calculating statistics is one of many band math operations involving single band. 
Other single band operations that we deal with in this course are thresholding. 
We shall discuss about thresholding in the later labs.

Let's read the red band raster and calculate its mean, median, max and min values.

**Note:** We are processing the downsampled bands you created in the prior lab, [Transforms](./Transforms.ipynb).

In [ ]:
########---------------------
### Reading pixel values from  red band
########---------------------
import rasterio
import numpy as np
from pprint import pprint

with rasterio.open('../temp/redband_downsampled.TIF') as src:
    redband = src.read()
stats = []
stats.append({
        'min': redband.min(),
        'mean': redband.mean(),
        'median': np.median(redband),
        'max': redband.max()})

pprint(stats)

From the statistics, it looks like the redband has a range of (63030), 
i.e., the values in the red band are in the range [0,57917].

Let's check the statistics of near-infrared band raster

In [ ]:
with rasterio.open('../temp/nearIRband_downsampled.TIF') as src:
    nirband = src.read()
stats = []
stats.append({
        'min': nirband.min(),
        'mean': nirband.mean(),
        'median': np.median(nirband),
        'max': nirband.max()})

pprint(stats)

From the statistics, it looks like the NIR band has a range of (62160), i.e., 
the values in the NIR band are in the range [0,62160].

So, the range is a bit different.
In some cases it is desirable to have the same range for both the bands.
Let's bring them into the range [0.0,1.0] using the below equation.

\begin{equation}
X' = \frac{X-min(X)}{max(X) - min(X)}
\end{equation}

The process of mapping an image from its inital range to [0,1] is referred to as _<b>nornalization_.

In [ ]:
redband = (redband - redband.min())/(redband.max() - redband.min())
nirband = (nirband - nirband.min())/(nirband.max() - nirband.min())

Now check the range of both the bands

In [ ]:
stats = []

stats.append({
        'band': 'RED',
        'min': redband.min(),
        'mean': redband.mean(),
        'median': np.median(redband),
        'max': redband.max()})


stats.append({
        'band': 'NIR',
        'min': nirband.min(),
        'mean': nirband.mean(),
        'median': np.median(nirband),
        'max': nirband.max()})
pprint(stats)

Now that we have seen how band math is performed on single bands, let's move on to band math involving multiple bands.

## Multiple Bands

This section will demonstrate the usefulness of band math involving multiple bands. 
Specifically, we will calculate _Normalized Difference Vegetation Index_ (NDVI), 
an indicator of biomass, from different bands of a remote sensing image. 

### Normalized Difference Vegetation Index (NDVI) 
Normalized Difference Vegetation index ([NDVI](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index)) is a simple graphical indicator used to assess presence of live green vegetation in the area under observation.
This calculation uses the Red and Near-Infrared (NIR) bands of a multispectral image dataset.

**Note:** Recall from early school days, green plants are reflecting green wavelength light and absorbing other wavelengths.  In essence, we can measure the amount of NIR radiation reflected versus Red reflected.  The more Red absorbed, the less reflected.

[Read more on NDVI](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index)

#### Calculating NDVI

Lets start out by
* Reading the pixel values from both the rasters
* Visualize bands
* Calculating NDVI
* Visualizing the NDVI values

##### Reading the pixel values from both the rasters

In [ ]:
########---------------------
### Reading pixel values from  red band
########---------------------
import rasterio

with rasterio.open('../temp/redband_downsampled.TIF') as band1:
    redband = band1.read()


########---------------------
### Reading pixel values from  near-infrared band
########---------------------
with rasterio.open('../temp/nearIRband_downsampled.TIF') as band2:
       nirband = band2.read()

In [ ]:
redband.max()

In [ ]:
redband.dtype

##### Visualize bands

In [ ]:
########---------------------
### Visualizing  red band
########---------------------
from rasterio.plot import show
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

fig = plt.figure(figsize= (6,6))
rasterio.plot.show(redband)

In [ ]:
########---------------------
### Visualizing  near-infrared band
########---------------------
from rasterio.plot import show


fig = plt.figure(figsize= (6,6))
rasterio.plot.show(nirband)

##### Calculating NDVI 

In [ ]:
nirband = nirband.astype(np.float64)
redband = redband.astype(np.float64)
ndvi = (nirband - redband)/(nirband + redband+0.0000000001) # +0.0000000001 is to avoid 0/0

##### Visualizing the NDVI values

We shall visualize the NDVI obtained using `rasterio.plot.show()` with `YlGn` as colormap. 
Save the NDVI image obtained for future purposes.


In [ ]:
%matplotlib notebook
%matplotlib notebook

fig = plt.figure(figsize=(6,6))
rasterio.plot.show(ndvi, cmap = 'YlGn')
ndvi.min()

Let's save the above image for future labs.

In [ ]:
src = rasterio.open('../temp/redband_downsampled.TIF')
kwargs = src.meta.copy()
kwargs.update({ 'dtype': rasterio.float64 })
print(kwargs)

In [ ]:
with rasterio.open('../temp/ndvi_downsampled_1.TIF','w', **kwargs) as dst:
     dst.write(ndvi)

In [ ]:
%matplotlib inline
with rasterio.open('../temp/ndvi_downsampled_1.TIF') as src:
    fig = plt.figure(figsize = (6,6))
    rasterio.plot.show(src.read(),cmap = 'YlGn')

### Analyzing NDVI image

The NDVI values range from -1 to 1. The greener the area, the more the presence of live green vegetation. 
In general, 
* Areas containing a dense vegetation will tend to have positive values (say 0.3 to 0.8) 
* Clouds and snow fields will be characterized by negative values of this index.
* Free standing water tend to have very low positive or even slightly negative NDVI values,
* Soils also tend to generate small positive NDVI values (say 0.1 to 0.2).

>According to the docs for [Landsat 8](https://landsat.usgs.gov/collectionqualityband), those blank areas around the edges should be ignored. Many raster datasets implement this with an optional nodata value.

Let's follow the above guidelines, and compare the original RGB image of the scene with the NDVI image.

In [ ]:
import matplotlib.pyplot as plt

%matplotlib notebook
%matplotlib notebook

original_image = ''
with rasterio.open('/dsa/data/all_datasets/rasters/RGB.byte.tif') as data:
    original_image = data.read()


fig = plt.figure(figsize=(10,10))
a = fig.add_subplot(1,2,1)
a.set_title('Original RGB Image')
rasterio.plot.show(original_image)
a = fig.add_subplot(1,2,2)
a.set_title('NDVI index image')
rasterio.plot.show(ndvi, cmap = 'YlGn')


If you observe the above images closely, the snow fields and clouds assume negative values, while water assumed small positive values. The area to the right of standing water has dense green vegetation.

> As you can imagine one interesting application of such an index is to determine the presence of life on other planets.

### Save the NDVI image as `ndvi_downsampled.TIF` following the reprojection procedure from previous `Transforms.ipynb` lab.

_Tips for saving NDVI_
    1. Normalize the NDVI image to the range [0,1], its original range would be [-1,1]
    2. After normalizing, convert it into the range [0,2^16 -1] by multiplying it with (2^16 -1)
    
Read the `ndvi_downsampled.tif` file and visualize the raster. It should look like below.

![NDVI_raster.png MISSING](../images/NDVI_raster.png)

In [ ]:
from rasterio.warp import reproject,RESAMPLING

ndvi = (ndvi - ndvi.min())/(ndvi.max() - ndvi.min() + 0.00000000001)
ndvi = (ndvi*(2**16 -1)).astype(np.uint16)
with rasterio.open('../temp/redband_downsampled.TIF') as src:
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': src.crs,
        'transform': src.transform,
        'width' : src.width,
        'height': src.height
    })

    with rasterio.open('../temp/ndvi_downsampled.TIF','w', **kwargs) as dst:
        reproject(
            source = ndvi, 
            destination = rasterio.band(dst,1),
            src_transform = src.transform,
            src_crs = src.crs,
            dst_transform = src.transform,
            dst_crs = src.crs,
            resampling = RESAMPLING.nearest)

In [ ]:
with rasterio.open('../temp/ndvi_downsampled.TIF') as ndvi_img:
    
    fig = plt.figure(figsize=(6,6))
    rasterio.plot.show(ndvi_img.read())

# Save your Notebook
## Then, File > Close and Halt